In [8]:
from pathlib import Path
from llama_index.core import SimpleDirectoryReader, load_index_from_storage, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.settings import Settings
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

CUTOFF = 0.6
MODEL_NAME = 'KBLab/sentence-bert-swedish-cased'
DATA_FOLDER = Path("bp2025")
INDEX_FOLDER = DATA_FOLDER / Path("indexes")


In [9]:

def get_metadata_from_filename(filename):
    """Custom function to fetch metadata from filename."""
    path=Path(filename)
    return {
        "document_type": path.parent.name,
        "file_name": str(path.name),
    }

# load data
loader = SimpleDirectoryReader(
    input_dir=DATA_FOLDER,
    required_exts=[".pdf"], # Läser bara in pdf-filer
    recursive=False,
    file_metadata=get_metadata_from_filename)


In [13]:

from torch import backends, cuda
if backends.mps.is_available():
    device = "mps"
    print("Using MPS")
else:
    # check for cuda
    if cuda.is_available():
        device = "cuda"
        print("Using CUDA")
    else:
        device = "cpu"
        print("Using CPU")
        
embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, trust_remote_code=True, device=device)

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=256, chunk_overlap=20),
        embed_model,
    ]
)

# Check if INDEX_FOLDER is empty. If empty run the pipeline and save the index
if not INDEX_FOLDER.exists() or not any(INDEX_FOLDER.iterdir()):

    # create index folder
    INDEX_FOLDER.mkdir(parents=True, exist_ok=True)

    print("No index found. Running pipeline and saving index.")

    docs = loader.load_data()
    print('Number of documents:', len(docs))
    
    # run the pipeline
    nodes = pipeline.run(documents=docs, show_progress=True)

    # Building index from nodes
    index = VectorStoreIndex(nodes, embed_model=embed_model)

    # Save index
    index.storage_context.persist(persist_dir=INDEX_FOLDER)

else:
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_FOLDER)
    # Load index
    index = load_index_from_storage(storage_context=storage_context,embed_model=embed_model)

    print("Index loaded.")

Using MPS


/Users/johve/source/semantic_search_llama_index/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Index loaded.


In [14]:
# Make a retriever object
retriever = index.as_retriever(similarity_top_k=5)

In [24]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)
target_text="""Hur går det för unga på arbetsmarknaden?"""

retrieved_results = retriever.retrieve(target_text)

for r in retrieved_results:
    print(f"Score: {r.score:.2f}")
    print(f"Page: {r.metadata['page_label']}")
    print(f"Name: {r.metadata['file_name']}")
    print(f'Text: {r.text}')
    print('-'*80)

Score: 0.74
Page: 2295
Name: budgetpropositionen-for-2025-hela-dokumentet-prop.-2024251.pdf
Text: Prop.  2024/25:1  Utgiftsområde  17 
150 Ungas förutsättningar för att etablera sig på arbetsmarknaden har under 2023 fortsatt 
att utvec klas i en positiv riktning. Andelen unga inskrivna på Arbetsförmedlingen har 
fortsatt att minska bland samtliga grupper av unga och nedgången har varit särskilt 
tydlig bland gruppen unga utrikes födda, vilket bidragit till att skillnaden mellan utrikes 
och inrikes födda unga har minskat. Sammantaget medför detta att arbetslösheten 
bland unga, mätt som andel unga inskrivna på Arbetsförmedlingen, är den lägsta på 
mer än ett decennium.
--------------------------------------------------------------------------------
Score: 0.73
Page: 150
Name: utgiftsomrade-17-kultur-medier-trossamfund-och-fritid.pdf
Text: Prop.  2024/25:1  Utgiftsområde  17 
150 Ungas förutsättningar för att etablera sig på arbetsmarknaden har under 2023 fortsatt 
att utvec klas i en p